In [1]:
import random
import requests
from datetime import datetime
import sqlite3
import json


In [2]:
def conectar_banco(nome_banco=fr"E:\Projetos\Jupyter\lotofacil.db"):
    """Estabelece conexão com o banco de dados SQLite3."""
    conexao = sqlite3.connect(nome_banco)
    cursor = conexao.cursor()
    return conexao, cursor


In [3]:
def getGanhadores(premiacoes):
    
    quinze = 0
    quatorze = 0
    treze = 0
    doze = 0
    onze = 0
    
    if premiacoes is None:
        return quinze, quatorze, treze, doze, onze        
    
    for premiacao in premiacoes:
        if premiacao['descricao'] == '15 acertos':
            quinze = premiacao['ganhadores']
        elif premiacao['descricao'] == '14 acertos':
            quatorze = premiacao['ganhadores']
        elif premiacao['descricao'] == '13 acertos':
            treze = premiacao['ganhadores']
        elif premiacao['descricao'] == '12 acertos':
            doze = premiacao['ganhadores']
        elif premiacao['descricao'] == '11 acertos':
            onze = premiacao['ganhadores']
            
    return quinze, quatorze, treze, doze, onze
            

In [6]:
def update(query):
    conexao, cursor = conectar_banco()                
    
    # Executar a consulta SQL
    cursor.execute(query)    
    conexao.commit()    
    conexao.close()
    
    return True

In [5]:
def select(query):
    conexao, cursor = conectar_banco()        
    
    # Criar o DataFrame a partir da consulta SQL
    # df = pd.read_sql_query(query, conexao)
    
    # Executar a consulta SQL
    cursor.execute(query)
    
    # Obter os nomes das colunas
    colunas = [descricao[0] for descricao in cursor.description]
    
    # Obter todos os resultados da consulta
    dados = cursor.fetchall()
    
    # Transformar os dados em uma lista de dicionários (JSON-friendly)
    resultado_json = [dict(zip(colunas, linha)) for linha in dados]        
    
    # Converter para JSON
    json_saida = json.dumps(resultado_json, indent=4, ensure_ascii=False)                
    
    conexao.close()
    
    return json.loads(json_saida)

In [9]:
def calcParesImpares(concurso):
    
    dados = select(f"select * from dataframe where concurso = {concurso}")[0]

    if dados['pares'] is None:
        
        pares = 0
        impares = 0                    
        
        if dados['dezena_1'] % 2 == 0:
            pares += 1
        
        if dados['dezena_2'] % 2 == 0:
            pares += 1

        if dados['dezena_3'] % 2 == 0:
            pares += 1
            
        if dados['dezena_4'] % 2 == 0:
            pares += 1
            
        if dados['dezena_5'] % 2 == 0:
            pares += 1   
            
        if dados['dezena_6'] % 2 == 0:
            pares += 1
        
        if dados['dezena_7'] % 2 == 0:
            pares += 1

        if dados['dezena_8'] % 2 == 0:
            pares += 1
            
        if dados['dezena_9'] % 2 == 0:
            pares += 1
            
        if dados['dezena_10'] % 2 == 0:
            pares += 1
            
        if dados['dezena_11'] % 2 == 0:
            pares += 1
        
        if dados['dezena_12'] % 2 == 0:
            pares += 1

        if dados['dezena_13'] % 2 == 0:
            pares += 1
            
        if dados['dezena_14'] % 2 == 0:
            pares += 1
            
        if dados['dezena_15'] % 2 == 0:
            pares += 1        
            
        impares = 15 - pares
        
        print(f'Concurso {concurso} Tem {pares} pares e {impares} impares')   
        
        upt = f"""
                update dataframe
                set pares = {pares}, impares = {impares}
                where concurso = {concurso}
            """    
        update(upt) 
    else:
        print(f'Concurso {concurso} já foi processado')   

In [ ]:
def obter_dados_reais(loteria, concurso):        
    
    url = f"https://loteriascaixa-api.herokuapp.com/api/{loteria}/{concurso}"
    
    try:
        resposta = requests.get(url)
        dados = resposta.json()        
        dezenas = dados["dezenas"]
        data = dados['data']
        data_formatada = datetime.strptime(data, "%d/%m/%Y").strftime("%Y-%m-%d")
        acumulou = dados['acumulou']
        premiacoes = dados['premiacoes']      
        quinze_acertos, quatorze_acertos, treze_acertos, doze_acertos, onze_acertos = getGanhadores(premiacoes)          
        dezena_1 = dezenas[0]
        dezena_2 = dezenas[1]
        dezena_3 = dezenas[2]
        dezena_4 = dezenas[3]
        dezena_5 = dezenas[4]
        dezena_6 = dezenas[5]
        dezena_7 = dezenas[6]
        dezena_8 = dezenas[7]
        dezena_9 = dezenas[8]
        dezena_10 = dezenas[9]
        dezena_11 = dezenas[10]
        dezena_12 = dezenas[11]
        dezena_13 = dezenas[12]
        dezena_14 = dezenas[13]
        dezena_15 = dezenas[14]                                           
        
        
        conexao, cursor = conectar_banco()
                
        query = """
        INSERT INTO dataframe (
            loteria, concurso, data, dezena_1, dezena_2, dezena_3, dezena_4, dezena_5, 
            dezena_6, dezena_7, dezena_8, dezena_9, dezena_10, dezena_11, 
            dezena_12, dezena_13, dezena_14, dezena_15, acumulou, quinze_acertos, 
            quatorze_acertos, treze_acertos, doze_acertos, onze_acertos
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        valores = (
            loteria, concurso, data_formatada, dezena_1, dezena_2, dezena_3, dezena_4, dezena_5,
            dezena_6, dezena_7, dezena_8, dezena_9, dezena_10,  dezena_11, dezena_12, dezena_13, dezena_14, dezena_15,
            acumulou, quinze_acertos, quatorze_acertos, treze_acertos, doze_acertos, onze_acertos)
        
        # Exibir a query formatada antes de executar
        # query_formatada = query.replace("?", "{}").format(*valores)
        # print("Query formatada:", query_formatada)
        print(f'+1 \t concurso: {concurso}')
        cursor.execute(query, valores)
        conexao.commit()     
        
        calcParesImpares(concurso)   
        
    except Exception as e:
        print("Erro ao obter dados reais:", e)        
    finally:
        # Fecha a conexão corretamente para evitar LOCK
        conexao.close()


In [ ]:
# Calcula pares e ímpares
for concurso in range(3333,3334):
    # obter_dados_reais('lotofacil', concurso)    

    dados = select(f"select * from dataframe where concurso = {concurso}")[0]

    if dados['pares'] is None:
        
        pares = 0
        impares = 0                    
        
        if dados['dezena_1'] % 2 == 0:
            pares += 1
        
        if dados['dezena_2'] % 2 == 0:
            pares += 1

        if dados['dezena_3'] % 2 == 0:
            pares += 1
            
        if dados['dezena_4'] % 2 == 0:
            pares += 1
            
        if dados['dezena_5'] % 2 == 0:
            pares += 1   
            
        if dados['dezena_6'] % 2 == 0:
            pares += 1
        
        if dados['dezena_7'] % 2 == 0:
            pares += 1

        if dados['dezena_8'] % 2 == 0:
            pares += 1
            
        if dados['dezena_9'] % 2 == 0:
            pares += 1
            
        if dados['dezena_10'] % 2 == 0:
            pares += 1
            
        if dados['dezena_11'] % 2 == 0:
            pares += 1
        
        if dados['dezena_12'] % 2 == 0:
            pares += 1

        if dados['dezena_13'] % 2 == 0:
            pares += 1
            
        if dados['dezena_14'] % 2 == 0:
            pares += 1
            
        if dados['dezena_15'] % 2 == 0:
            pares += 1        
            
        impares = 15 - pares
        
        print(f'Concurso {concurso} Tem {pares} pares e {impares} impares')   
        
        upt = f"""
                update dataframe
                set pares = {pares}, impares = {impares}
                where concurso = {concurso}
            """    
        update(upt) 
    else:
        print(f'Concurso {concurso} já foi processado')                             


Concurso 1 já foi processado
Concurso 2 já foi processado
Concurso 3 já foi processado
Concurso 4 já foi processado
Concurso 5 Tem 7 pares e 8 impares
Concurso 6 Tem 6 pares e 9 impares
Concurso 7 Tem 8 pares e 7 impares
Concurso 8 Tem 7 pares e 8 impares
Concurso 9 Tem 5 pares e 10 impares
Concurso 10 Tem 9 pares e 6 impares
Concurso 11 Tem 9 pares e 6 impares
Concurso 12 Tem 8 pares e 7 impares
Concurso 13 Tem 5 pares e 10 impares
Concurso 14 Tem 5 pares e 10 impares
Concurso 15 Tem 9 pares e 6 impares
Concurso 16 Tem 6 pares e 9 impares
Concurso 17 Tem 6 pares e 9 impares
Concurso 18 Tem 9 pares e 6 impares
Concurso 19 Tem 8 pares e 7 impares
Concurso 20 Tem 9 pares e 6 impares
Concurso 21 Tem 9 pares e 6 impares
Concurso 22 Tem 7 pares e 8 impares
Concurso 23 Tem 9 pares e 6 impares
Concurso 24 Tem 5 pares e 10 impares
Concurso 25 Tem 8 pares e 7 impares
Concurso 26 Tem 6 pares e 9 impares
Concurso 27 Tem 9 pares e 6 impares
Concurso 28 Tem 6 pares e 9 impares
Concurso 29 Tem 8 par

In [10]:
# obter_dados_reais('lotofacil', 3333)    
calcParesImpares(3333)

Concurso 3333 Tem 5 pares e 10 impares


In [63]:
def generate_sql_query():
    columns = [f"dezena_{i}" for i in range(1, 16)]  # Gera ['dezena_1', ..., 'dezena_15']
    query_parts = []

    # Gerar todas as combinações possíveis de pares
    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            query_parts.append(f"SELECT {columns[i]}, {columns[j]} FROM dataframe")

    # Unir todas as queries com UNION ALL
    query_union = " UNION ALL\n".join(query_parts)

    # Criar a query final
    query = f"""
WITH pares AS (
    {query_union}
)
SELECT printf("%d,%d", MIN(dezena_1, dezena_2), MAX(dezena_1, dezena_2)) AS descricao,
       COUNT(*) AS quantidade
FROM pares
GROUP BY descricao
ORDER BY quantidade DESC;
"""
    return query

# Gerar e imprimir a query
sql_query = generate_sql_query()
print(sql_query)



WITH pares AS (
    SELECT dezena_1, dezena_2 FROM dataframe UNION ALL
SELECT dezena_1, dezena_3 FROM dataframe UNION ALL
SELECT dezena_1, dezena_4 FROM dataframe UNION ALL
SELECT dezena_1, dezena_5 FROM dataframe UNION ALL
SELECT dezena_1, dezena_6 FROM dataframe UNION ALL
SELECT dezena_1, dezena_7 FROM dataframe UNION ALL
SELECT dezena_1, dezena_8 FROM dataframe UNION ALL
SELECT dezena_1, dezena_9 FROM dataframe UNION ALL
SELECT dezena_1, dezena_10 FROM dataframe UNION ALL
SELECT dezena_1, dezena_11 FROM dataframe UNION ALL
SELECT dezena_1, dezena_12 FROM dataframe UNION ALL
SELECT dezena_1, dezena_13 FROM dataframe UNION ALL
SELECT dezena_1, dezena_14 FROM dataframe UNION ALL
SELECT dezena_1, dezena_15 FROM dataframe UNION ALL
SELECT dezena_2, dezena_3 FROM dataframe UNION ALL
SELECT dezena_2, dezena_4 FROM dataframe UNION ALL
SELECT dezena_2, dezena_5 FROM dataframe UNION ALL
SELECT dezena_2, dezena_6 FROM dataframe UNION ALL
SELECT dezena_2, dezena_7 FROM dataframe UNION ALL
SELE